In [4]:
#物体识别
import numpy as np
import sklearn.svm as svm
import sklearn.metrics as sm
import os
import sklearn.preprocessing as sp   #预处理
import cv2 as cv

def search_files(directory):
    objects={}
    for curdir,subdirs,files in os.walk(directory):    
        for file in files:
            if file.endswith('.jpg'):
                label=curdir.split(os.path.sep)[-1]  #curdir当前目录
                if label not in objects:
                    objects[label]=[]
                url=os.path.join(curdir,file)
                objects[label].append(url)
    return objects            
train_urls=search_files('../ml_data/objects/training')
print(train_urls)      

{'airplane': ['../ml_data/objects/training\\airplane\\0001.jpg', '../ml_data/objects/training\\airplane\\0002.jpg', '../ml_data/objects/training\\airplane\\0003.jpg', '../ml_data/objects/training\\airplane\\0004.jpg', '../ml_data/objects/training\\airplane\\0005.jpg', '../ml_data/objects/training\\airplane\\0006.jpg', '../ml_data/objects/training\\airplane\\0007.jpg', '../ml_data/objects/training\\airplane\\0008.jpg', '../ml_data/objects/training\\airplane\\0009.jpg', '../ml_data/objects/training\\airplane\\0010.jpg', '../ml_data/objects/training\\airplane\\0011.jpg', '../ml_data/objects/training\\airplane\\0012.jpg', '../ml_data/objects/training\\airplane\\0013.jpg', '../ml_data/objects/training\\airplane\\0014.jpg', '../ml_data/objects/training\\airplane\\0015.jpg', '../ml_data/objects/training\\airplane\\0016.jpg', '../ml_data/objects/training\\airplane\\0017.jpg', '../ml_data/objects/training\\airplane\\0018.jpg', '../ml_data/objects/training\\airplane\\0019.jpg'], 'car': ['../ml_d

In [17]:
train_x,train_y=[],[]
for label,urls in train_urls.items():
    for file in urls:
        #提取file图片的特征值矩阵，整理样本
        image=cv.imread(file)
        gray=cv.cvtColor(image,cv.COLOR_BGR2GRAY)
        h,w=gray.shape
        f=200/min(h,w)#计算伸缩比例，让较小的边==200，较大的边自适应
        gray=cv.resize(gray,None,fx=f,fy=f)
        sift=cv.xfeatures2d.SIFT_create()
        keypoints=sift.detect(gray)
        #desc :(n,128)特征值矩阵
        _,desc=sift.compute(gray,keypoints)
        sample=np.mean(desc,axis=0)
        train_x.append(sample)
        train_y.append(label)
train_x=np.array(train_x)    
encoder=sp.LabelEncoder()
train_y_label=encoder.fit_transform(train_y)


In [22]:
#训练svm 
model=svm.SVC(kernel='poly',degree=2,probability=True)
import sklearn.linear_model as lm
model=lm.LogisticRegression()
model.fit(train_x,train_y_label)
pred_train_y=model.predict(train_x)
print(sm.classification_report(train_y_label,pred_train_y))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        19
           2       1.00      1.00      1.00        19

    accuracy                           1.00        57
   macro avg       1.00      1.00      1.00        57
weighted avg       1.00      1.00      1.00        57



C:\Users\88\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [23]:
#置信概率
probs=model.predict_proba(train_x)
print(np.round(probs,2))
for label,prob in zip(
        encoder.inverse_transform(pred_train_y),
        probs.max(axis=1)):
    print(label,prob)

[[1.   0.   0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [0.98 0.01 0.01]
 [0.99 0.   0.01]
 [1.   0.   0.  ]
 [0.99 0.   0.01]
 [0.98 0.01 0.01]
 [0.99 0.   0.01]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [1.   0.   0.  ]
 [0.99 0.01 0.  ]
 [1.   0.   0.  ]
 [0.99 0.01 0.  ]
 [0.99 0.   0.01]
 [0.99 0.01 0.  ]
 [0.98 0.02 0.  ]
 [0.   1.   0.  ]
 [0.   1.   0.  ]
 [0.   0.99 0.  ]
 [0.   1.   0.  ]
 [0.01 0.99 0.  ]
 [0.01 0.99 0.  ]
 [0.   0.99 0.01]
 [0.   1.   0.  ]
 [0.01 0.99 0.  ]
 [0.   1.   0.  ]
 [0.   1.   0.  ]
 [0.01 0.99 0.  ]
 [0.   1.   0.  ]
 [0.01 0.99 0.  ]
 [0.   1.   0.  ]
 [0.   1.   0.  ]
 [0.   1.   0.  ]
 [0.01 0.99 0.  ]
 [0.   1.   0.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.01 0.   0.99]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.01 0.   0.99]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.01 0.   0.99]
 [0.01 0.   0.99]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0.   1.  ]
 [0.   0. 